In [1]:
import ee
ee.Initialize()

In [2]:
geometry = ee.Geometry.Point([-100.55, 40.71])
dataset = ee.Image('CGIAR/SRTM90_V4')
elevation = dataset.select('elevation')

# Multiscale

In [3]:
def create_A_matrix(windowSize, cellSize):
    A = [[0 for x in range(5)] for y in range(5)]  # 5 x 5 matrix
    
    midCell = (windowSize - 1) / 2
    
    x2 = 0
    x4 = 0
    x2y2 = 0
    
    for i in range(windowSize):
        for j in range(windowSize):
            Xpos = (i- midCell) * cellSize
            Ypos = (j - midCell) * cellSize
            x2 += Xpos ** 2
            x4 += Xpos ** 4
            x2y2 += Xpos ** 2 * Ypos ** 2
    
    A[0][0] = x4
    A[1][1] = x4
    A[0][1] = x2y2
    A[1][0] = x2y2
    A[2][2] = x2y2
    A[3][3] = x2
    A[4][4] = x2
    
    return A

In [4]:
cellSize = elevation.projection().nominalScale().getInfo()

print(cellSize)
A = create_A_matrix(5, cellSize)

92.76624232769086


In [5]:
A

[[12589508677.476757, 7405593339.69221, 0, 0, 0],
 [7405593339.69221, 12589508677.476757, 0, 0, 0],
 [0, 0, 7405593339.69221, 0, 0],
 [0, 0, 0, 430278.78577999317, 0],
 [0, 0, 0, 0, 430278.78577999317]]

In [6]:
import numpy as np
A_inv = np.linalg.inv(A)

In [7]:
A_inv

array([[ 1.21458316e-10, -7.14460685e-11,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-7.14460685e-11,  1.21458316e-10,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.35033069e-10,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         2.32407461e-06,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.32407461e-06]])

In [8]:
def create_kernel(windowSize):
    weight_val = 1
    weights = ee.List.repeat(ee.List.repeat(weight_val, windowSize), windowSize)
    
    midCell = ee.Number(windowSize).subtract(1).divide(2)
    
    x = midCell.multiply(-1)
    y = midCell.multiply(-1)
  
    mid_row = ee.List.repeat(weight_val, midCell)\
        .cat([0])\
        .cat(ee.List.repeat(weight_val, midCell))
  
    weights = weights.set(midCell, mid_row)

    kernel = ee.Kernel.fixed(windowSize, windowSize, weights, x, y, False)   
  
    return kernel

In [9]:
kernel = create_kernel(5)

In [10]:
print(kernel.getInfo())

{'type': 'Kernel.fixed', 'width': 5, 'height': 5, 'weights': '\n  [1.0, 1.0, 1.0, 1.0, 1.0]\n  [1.0, 1.0, 1.0, 1.0, 1.0]\n  [1.0, 1.0, 0.0, 1.0, 1.0]\n  [1.0, 1.0, 1.0, 1.0, 1.0]\n  [1.0, 1.0, 1.0, 1.0, 1.0]', 'x': 2, 'y': 2, 'center': [2, 2]}


In [11]:
neighs = elevation.neighborhoodToBands(kernel).updateMask(elevation.gt(0))

In [12]:
print(neighs.bandNames().getInfo())

['elevation_-2_-2', 'elevation_-1_-2', 'elevation_0_-2', 'elevation_1_-2', 'elevation_2_-2', 'elevation_-2_-1', 'elevation_-1_-1', 'elevation_0_-1', 'elevation_1_-1', 'elevation_2_-1', 'elevation_-2_0', 'elevation_-1_0', 'elevation_1_0', 'elevation_2_0', 'elevation_-2_1', 'elevation_-1_1', 'elevation_0_1', 'elevation_1_1', 'elevation_2_1', 'elevation_-2_2', 'elevation_-1_2', 'elevation_0_2', 'elevation_1_2', 'elevation_2_2']


In [62]:
import math
import re

label = neighs.bandNames().getInfo()
Z0 = ee.Image(0).rename('Z0').updateMask(elevation.gt(0))
Z1 = ee.Image(0).rename('Z1').updateMask(elevation.gt(0))
Z2 = ee.Image(0).rename('Z2').updateMask(elevation.gt(0))
Z3 = ee.Image(0).rename('Z3').updateMask(elevation.gt(0))
Z4 = ee.Image(0).rename('Z4').updateMask(elevation.gt(0))

for r in label:
    y = [int(s) for s in re.findall(r'-?\d+\.?\d*',r)][0] * cellSize
    x = [int(s) for s in re.findall(r'-?\d+\.?\d*',r)][1] * cellSize
    Z0 = Z0.add(neighs.select(r).subtract(elevation).multiply(x ** 2))
    Z1 = Z1.add(neighs.select(r).subtract(elevation).multiply(y ** 2))
    Z2 = Z2.add(neighs.select(r).subtract(elevation).multiply(x * y))
    Z3 = Z3.add(neighs.select(r).subtract(elevation).multiply(x))
    Z4 = Z4.add(neighs.select(r).subtract(elevation).multiply(y))

In [83]:
# R
temp = ee.Image(0).rename('temp').updateMask(elevation.gt(0))
temp = temp.add(Z0.multiply(A_inv[0][0]))
temp = temp.add(Z1.multiply(A_inv[0][1]))
temp = temp.add(Z2.multiply(A_inv[0][2]))
temp = temp.add(Z3.multiply(A_inv[0][3]))
temp = temp.add(Z4.multiply(A_inv[0][4]))

R = ee.Image(temp).rename('R').updateMask(elevation.gt(0)) 

# T
temp = ee.Image(0).rename('temp').updateMask(elevation.gt(0))
temp = temp.add(Z0.multiply(A_inv[1][0]))
temp = temp.add(Z1.multiply(A_inv[1][1]))
temp = temp.add(Z2.multiply(A_inv[1][2]))
temp = temp.add(Z3.multiply(A_inv[1][3]))
temp = temp.add(Z4.multiply(A_inv[1][4]))

T = ee.Image(temp).rename('T').updateMask(elevation.gt(0)) 

# S
temp = ee.Image(0).rename('temp').updateMask(elevation.gt(0))
temp = temp.add(Z0.multiply(A_inv[2][0]))
temp = temp.add(Z1.multiply(A_inv[2][1]))
temp = temp.add(Z2.multiply(A_inv[2][2]))
temp = temp.add(Z3.multiply(A_inv[2][3]))
temp = temp.add(Z4.multiply(A_inv[2][4]))

S = ee.Image(temp).rename('S').updateMask(elevation.gt(0)) # 0

# P
temp = ee.Image(0).rename('temp').updateMask(elevation.gt(0))
temp = temp.add(Z0.multiply(A_inv[3][0]))
temp = temp.add(Z1.multiply(A_inv[3][1]))
temp = temp.add(Z2.multiply(A_inv[3][2]))
temp = temp.add(Z3.multiply(A_inv[3][3]))
temp = temp.add(Z4.multiply(A_inv[3][4]))

P = ee.Image(temp).rename('P').updateMask(elevation.gt(0)) # 0

# Q
temp = ee.Image(0).rename('temp').updateMask(elevation.gt(0))
temp = temp.add(Z0.multiply(A_inv[4][0]))
temp = temp.add(Z1.multiply(A_inv[4][1]))
temp = temp.add(Z2.multiply(A_inv[4][2]))
temp = temp.add(Z3.multiply(A_inv[4][3]))
temp = temp.add(Z4.multiply(A_inv[4][4]))

Q = ee.Image(temp).rename('Q').updateMask(elevation.gt(0)) # 0

        

In [85]:
gradient = ee.Image(0).expression('((p * p) + (q * q)) ** 0.5', {
      'p': P,
      'q': Q
    }).rename('gradient')


slope = gradient.atan().multiply(180/math.pi).rename('slope')

In [106]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.


# Create a folium map object.
my_map = folium.Map(location=[40.71, -100.55])

# Add the elevation model to the map object.
#my_map.add_ee_layer(elevation.updateMask(elevation.gt(0)), {'min':0, 'max':4000}, 'DEM')
my_map.add_ee_layer(slope, {'min':0, 'max':90}, 'slope')


# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [89]:
geometry1 = ee.Geometry.Point([-100.55, 40.71])

print(slope.sample(geometry1, 30).first().get('slope').getInfo())


0.2620389749960519
